In [2]:

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext import datasets, vocab
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

import numpy as np

from torch.utils.tensorboard import SummaryWriter

import random, tqdm, sys, math, gzip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("bookcorpus")

Dataset bookcorpus downloaded and prepared to /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


In [37]:
small_dataset = dataset['train'].select(range(1000000))

In [38]:
print(small_dataset)
small_dataset = small_dataset.train_test_split(test_size=0.1)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [39]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

small_dataset = small_dataset.map(lambda x: tokenizer(x['text'], padding='max_length', truncation=True, max_length=50, return_tensors='pt'), batched=True)

In [62]:
class TextDataset(Dataset):
    def __init__(self, dataset, seq_length):
        self.dataset = dataset
        self.seq_length = seq_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        input_ids = self.dataset[index]['input_ids']
        input_sequence = input_ids[:self.seq_length]
        target_sequence = input_ids[1:self.seq_length+1]

        input_sequence = torch.tensor(input_sequence, dtype=torch.long)
        target_sequence = torch.tensor(target_sequence, dtype=torch.long)

        return input_sequence, target_sequence

batch_size = 32
seq_length = 8
lr = 1e-4

train_dataset = TextDataset(small_dataset['train'], seq_length)
test_dataset = TextDataset(small_dataset['test'], seq_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [64]:

# batch_size = 32

# small_dataset.set_format(type="torch", columns=["input_ids"])
# train_loader = DataLoader(small_dataset['train'], batch_size=batch_size, shuffle=True)
# train_loader = DataLoader(small_dataset['test'], batch_size=batch_size, shuffle=True)

input_seq, target_seq  = next(iter(train_loader))
# print(input_seq)
which  = 4
print(tokenizer.decode(input_seq[which]))
print(tokenizer.decode(target_seq[which]))
# model
# print(tokenizer.decode(next(iter(train_loader))['input_ids'][0]))
# print(tokenizer.decode(next(iter(train_loader))['input_ids'][1]))

[CLS] i look up at the large tv
i look up at the large tv screen


In [66]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import importlib
import my_transformers
importlib.reload(my_transformers)

model = my_transformers.GenerationTransformer(k=4, heads=4, depth=2, seq_length=32, num_tokens=tokenizer.vocab_size).to(device)

num_epochs = 1
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()

    for input_sequence, target_sequence in tqdm.tqdm(train_loader):
        input_sequence = input_sequence.to(device)
        target_sequence = target_sequence.to(device)

        output = model(input_sequence)

        loss = loss_fn(
            output.view(-1, output.shape[-1]),
            target_sequence.view(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



100%|████████████████████████████████████| 28125/28125 [11:51<00:00, 39.50it/s]


In [106]:

for input_sequence, target_sequence in tqdm.tqdm(test_loader):
    input_sequence = input_sequence.to(device)
    # target_sequence = target_sequence.to(device)
    which = 24

    output = model(input_sequence)
    print(output.shape)
    next_token  = torch.argmax(output, dim=2)

    print(next_token.shape)

    print(tokenizer.decode(input_seq[which]))
    print(tokenizer.decode(target_seq[which]))
    print(next_token.shape)
    print(tokenizer.decode(next_token[which]))

    break
# input_seq, target_seq  = next(iter(train_loader))

# print(model(input_seq))
# # print(input_seq)
# which  = 4
# print(tokenizer.decode(input_seq[which]))
# print(tokenizer.decode(target_seq[which]))

  0%|                                                 | 0/3125 [00:00<?, ?it/s]

torch.Size([32, 8, 30522])
torch.Size([32, 8])
[CLS] the sound of his cell phone filled
the sound of his cell phone filled the
torch.Size([32, 8])
` `'` ` t,,
